In [0]:
def argmax_reject_threshold(y, threshold):
    y_argmax = np.argmax(y, axis=1)
    y_masked = np.ma.array(y_argmax, mask=(np.amax(y, axis=1) < threshold))

    return y_masked.filled(-1)

def argmax_top2_reject_threshold(y, threshold):
    y_argmax = np.argmax(y, axis=1)
    y_top2 = np.sort(y, axis=1)[:,-1:-3:-1]
    y_masked = np.ma.array(y_argmax, mask=((y_top2[:,0] - y_top2[:,1]) < threshold))

    return y_masked.filled(-1)

In [0]:
from sklearn.datasets import load_digits
digits = load_digits()

In [0]:
import matplotlib.pyplot as plt
plt.gray()
plt.matshow(digits.images[0])
plt.show()

In [0]:
X = digits.data
y = digits.target

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
from sklearn.neural_network import MLPClassifier
C = 10
clf1 = MLPClassifier(hidden_layer_sizes=C, activation='tanh', solver='sgd', batch_size=1, alpha=0, learning_rate='adaptive', verbose=1).fit(X_train, y_train)


In [0]:
from sklearn.model_selection import validation_curve
import numpy as np

C = np.array([1,2,3,4,5,8,10,15,20,25,30,35,40])

train_scores_list = []
validation_scores_list = []

for i in range(10):
    train_scores, validation_scores = validation_curve(
            MLPClassifier(activation='tanh', solver='adam', batch_size=1,alpha=0, learning_rate='adaptive',early_stopping=True, validation_fraction=0.2), 
            X_train, y_train, param_name="hidden_layer_sizes",param_range=C,scoring="accuracy", n_jobs=2)
    train_scores_list.append(train_scores.mean())
    validation_scores_list.append(validation_scores.mean())

train_mean = np.mean(train_scores_list)
train_std = np.std(train_scores_list)
validation_mean = np.mean(validation_scores_list)
validation_std = np.std(validation_scores_list)
    

In [0]:

plt.plot(C, train_scores_list, label='Train Scores')
plt.plot(C, validation_scores_list, label='Validation Scores')
plt.fill_between(C, train_mean-train_std, train_mean+train_std, alpha=0.2)
plt.fill_between(C, validation_mean-validation_std, validation_mean+validation_std, alpha=0.2)
plt.legend()
plt.show()

In [0]:
from sklearn.metrics import confusion_matrix
y_pred = clf1.predict(X_test)

conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

In [0]:
y_proba = clf1.predict_proba(X_train)


for threshold in np.arange(0, 1.01, 0.01):
    y_pred = argmax_reject_threshold(y_proba, threshold)
    rejection_rate = np.mean(y_pred == -1)
    recognition_rate = np.mean(y_pred[y_pred != -1] == y_train[y_pred != -1])
    print(f"Seuil: {threshold:.2f}, Taux de rejection: {rejection_rate:.4f}, Taux d'acceptation: {recognition_rate:.4f}")
    
for threshold in np.arange(0, 1.01, 0.01):
    y_pred = argmax_top2_reject_threshold(y_proba, threshold)
    rejection_rate = np.mean(y_pred == -1)
    recognition_rate = np.mean(y_pred[y_pred != -1] == y_train[y_pred != -1])

    print(f"Seuil: {threshold:.2f}, Taux de rejection: {rejection_rate:.4f}, Taux d'acceptation: {recognition_rate:.4f}")

In [0]:
reject_rates_1 = []
accept_rates_1 = []
reject_rates_2 = []
accept_rates_2 = []

for threshold in np.arange(0, 1.01, 0.01):
    y_pred = argmax_reject_threshold(y_proba, threshold)
    rejection_rate = np.mean(y_pred == -1)
    recognition_rate = np.mean(y_pred[y_pred != -1] == y_train[y_pred != -1])
    reject_rates_1.append(rejection_rate)
    accept_rates_1.append(recognition_rate)

for threshold in np.arange(0, 1.01, 0.01):
    y_pred = argmax_top2_reject_threshold(y_proba, threshold)
    rejection_rate = np.mean(y_pred == -1)
    recognition_rate = np.mean(y_pred[y_pred != -1] == y_train[y_pred != -1])
    reject_rates_2.append(rejection_rate)
    accept_rates_2.append(recognition_rate)


In [0]:
import matplotlib.pyplot as plt

plt.plot(reject_rates_1, accept_rates_1)
plt.plot(reject_rates_2, accept_rates_2)
plt.xlabel('Taux de rejection')
plt.ylabel("Taux d'acceptation")
plt.legend()
plt.show()


In [0]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

reject_indices = np.where(y_pred == -1)[0]
rejected_examples = X_train[reject_indices]
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
rejected_labels = knn.predict(rejected_examples)
y_pred[reject_indices] = rejected_labels


In [0]:

confusion_matrix_1 = confusion_matrix(y_train, clf1.predict(X_train))
print("Matrice Confusion (Premier etage):\n", confusion_matrix_1)

confusion_matrix_cascade = confusion_matrix(y_train, y_pred)
print("Matrice Confusion Matrix (Cascade):\n", confusion_matrix_cascade)
